In [ ]:
import json
import glob
import re
from functools import cache
from itertools import product
from os.path import join

import dateparser

In [ ]:
# URLs
with open('list.csv') as f:
    URLs = [link.strip() for link in list(f.readlines())[1:]]

In [ ]:
# get all the json files in the folder
json_files = list(sorted(glob.glob(join("data", "*.json"))))

# create a dict to store data
main_cases = []

# loop through and convert to csv
for file in json_files:
    with open(file) as f:
        data = json.load(f)
        main_cases.extend(data.values())
        
print(json.dumps(main_cases[:2], indent=2))

In [ ]:
# The data comes in as a list of cases with multiple people per case
# This method will convert the cases to a list of people with a unique key for each person as the index
# Use a list of people to do logic with, and then convert back to cases for the front end
def flatten_cases_to_people(cases):
    people = {}
    
    for case in cases:
        case_without_persons = case.copy()
        case_without_persons.pop('PersonsData')
        
        for person in case['PersonsData']:
            key = case['CaseRef'] + '%%' + person['Name']
            
            people[key] = person['InfoSection'] | case_without_persons
            people[key]['Name'] = person['Name']
            
    return people


def get_subset_of_cases(cases, people_keys):
    case_subset = []
    
    case_refs = {pk.split('%%')[0]: pk.split('%%')[1] for pk in people_keys}

    for case in cases:
        if case['CaseRef'] in case_refs:
            case_subset.append(case)
            
    return case_subset

In [ ]:
unique_attributes = set()

for row in main_cases:
    for person in row['PersonsData']:
        unique_attributes.update(list(person['InfoSection'].keys()))
        
print(unique_attributes)

# Unique Attributes Useful for Filtering
- **Discovered on**
- **Missing since**
- Year of birth
- Est. Age
- Age at disappearance
- Complexion
- Eye colour
- Height

In [ ]:
def format_age_range(raw_age_range_string):
    result = re.search(r'(\d+).*(\d+)', raw_age_range_string)
    
    if result is None or len(result.groups()) != 3:
        return None
    
    return [int(result.group(1)), int(result.group(2))]

In [ ]:
@cache
def cached_datetime_parser(*args, **kwargs):
    return dateparser.parse(*args, **kwargs)

In [ ]:
def person_data(data):

In [ ]:
def was_found_after_reported(unidentified, missing) -> bool:
    # TODO
    return True

In [ ]:
%%time

potential_matches_complete_objects = []

comparison_functions = [was_found_after_reported]

for unidentified, missing in product(unidentified_cases, missing_cases):
    # do comparisons of missing and unidentified using all known comparison functions
    # If it's a possible match, append to the list of potential matches
    if all([f(unidentified, missing) for f in comparison_functions]):
        potential_matches_complete_objects.append((unidentified, missing))
        
print(json.dumps(potential_matches_complete_objects[0], indent=2))

In [ ]:
def extract_attribute(cases, attribute, is_case_attribute=False):
    attribute_hash = {}
    
    for case in cases:
        for person in case['PersonsData']:
            key = case['CaseRef'] + '-' + person['Name']
            
            if is_case_attribute:
                data_section = case
            else:
                data_section = person['InfoSection']
            
            if attribute in data_section:
                attribute_hash[key] = data_section[attribute]
                
    return attribute_hash

In [ ]:

        

# print(list(flatten_cases_to_people(main_cases).items())[54])
print(get_subset_of_cases(main_cases, ['Case reference: 2014004727%%BELANGER, Roger Leo']))